# Preparations

## Mounting the Drive

In [150]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [151]:
DataFolder_Path = "/content/gdrive/MyDrive/EETBS /Data"

## Importing Modules

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [153]:
ConsumptionFileName= "consumption_5545.csv"
TemperatureFileName= "Austin_weather_2014.csv"
IrradianceFileName= "irradiance_2014_gen.csv"

## Files

In [154]:
ConsumptionFilePath = os.path.join(DataFolder_Path,ConsumptionFileName)
TemperatureFilePath  = os.path.join(DataFolder_Path,TemperatureFileName)
IrradianceFilePath = os.path.join(DataFolder_Path,IrradianceFileName)

In [155]:
DF_consumption = pd.read_csv(ConsumptionFilePath,sep=",", index_col=0, parse_dates=[0])

DF_weather = pd.read_csv(TemperatureFilePath,sep=";",index_col = ["localhour"],parse_dates=["localhour"],
                         date_parser= lambda col: pd.to_datetime(col, utc=True).tz_convert("US/Central").tz_localize(None)) 
DF_temperature = DF_weather[["temperature"]]

DF_irradiance = pd.read_csv(IrradianceFilePath,sep=";", index_col=["localhour"], parse_dates=["localhour"])
DF_irradiance = DF_irradiance[["gen"]]
DF_irradiance.loc[DF_irradiance["gen"]<0,"gen"]=0

In [156]:
DF_joined  = DF_consumption.join([DF_temperature,DF_irradiance])

DF_joined = DF_joined.rename(columns ={"air conditioner_5545":"Consumption [W]","temperature":"Temperature [°K]","gen":"Irradiance [??]"})

In [157]:
DF_joined.dropna(inplace = True)

In [158]:
Joined_DF_FileName= "joined_DF.csv"
RamosFolder_Path = "/content/gdrive/MyDrive/EETBS /Data/Ramos"
Joined_DF_FilePath = os.path.join(RamosFolder_Path,Joined_DF_FileName)
DF_joined.to_csv(Joined_DF_FilePath)

In [159]:
DF_mod = DF_joined.copy() 

# Assignment 12

## Functions

In [160]:
def laggedColumnCreator(df,ColumnName,LagStart,LagEnd,LagInterval):
  for i in list(range(LagStart,LagEnd+1,LagInterval)):
    NewColumnName = ColumnName + " -" + str(i) + "h"
    df[NewColumnName]= df[ColumnName].shift(i)
    df.dropna(inplace=True) #This applies it to the dataframe; if not you should use df=df.dropna()
  return df

In [161]:
def NegLaggedColumnCreator(df,ColumnName,LagStart,LagEnd,LagInterval):
  for i in list(range(LagStart,LagEnd+1,LagInterval)):
    NewColumnName = ColumnName + " +" + str(i) + "h"
    df[NewColumnName]= df[ColumnName].shift(-i)
    df.dropna(inplace=True) #This applies it to the dataframe; if not you should use df=df.dropna()
  return df

In [162]:
def Function(df): #This function assumes the index to be the time stamp
  df["Hours"] = df.index.hour
  df["sin(hour)"] = np.sin(df.index.hour*2*np.pi/24)
  df["cos(hour)"] = np.cos(df.index.hour*2*np.pi/24)
  df = NegLaggedColumnCreator(df,"cos(hour)",1,9,1)
  df["Day of the Year"] = df.index.dayofyear
  df["neg cos(DoY)"] = -np.cos((df.index.dayofyear-21)*2*np.pi/365)
  df["Day of the Week"] = df.index.isocalendar().day # Monday is 1
  df["Month"] = df.index.month
  df["Week of the Year"] = df.index.isocalendar().week # I don't understand why but this function considers the last three days of the years as part of the first week of the following year
  df["Weekend Flag"] = np.where(df.index.dayofweek<5,0,1)
  df["Working Hour"] = np.where(((df.index.hour>8) & (df.index.hour<19)),1,0)
  df["Summer Holidays"] = np.where(((df.index>"2014-06-15 00:00:00") & (df.index<"2014-09-05 00:00:00")),1,0)
  df["Night Time"] = np.where((((df.index.hour>=0)&(df.index.hour<=5))|((df.index.hour>=23)&(df.index.hour<=18))),0,1)
  df["Deep Night"] = np.where(((df.index.hour>=0) & (df.index.hour<=5)),1,0)
  df["Early Morning"] = np.where(((df.index.hour>=6) & (df.index.hour<=12)),1,0)
  df.dropna(inplace=True)
  return df

## Lags of Consumption Temperature and Irradiance

In [163]:
DF_mod = laggedColumnCreator(DF_mod,"Consumption [W]",1,24,1)
DF_mod = laggedColumnCreator(DF_mod,"Temperature [°K]",1,6,1)
DF_mod = laggedColumnCreator(DF_mod,"Irradiance [??]",3,12,1)

## Adding Time-Related Features

In [164]:
DF_mod = Function(DF_mod)

In [165]:
DF_mod.head()

,Consumption [W],Temperature [°K],Irradiance [??],Consumption [W] -1h,Consumption [W] -2h,Consumption [W] -3h,Consumption [W] -4h,Consumption [W] -5h,Consumption [W] -6h,Consumption [W] -7h,Consumption [W] -8h,Consumption [W] -9h,Consumption [W] -10h,Consumption [W] -11h,Consumption [W] -12h,Consumption [W] -13h,Consumption [W] -14h,Consumption [W] -15h,Consumption [W] -16h,Consumption [W] -17h,Consumption [W] -18h,Consumption [W] -19h,Consumption [W] -20h,Consumption [W] -21h,Consumption [W] -22h,Consumption [W] -23h,Consumption [W] -24h,Temperature [°K] -1h,Temperature [°K] -2h,Temperature [°K] -3h,Temperature [°K] -4h,Temperature [°K] -5h,Temperature [°K] -6h,Irradiance [??] -3h,Irradiance [??] -4h,Irradiance [??] -5h,Irradiance [??] -6h,Irradiance [??] -7h,Irradiance [??] -8h,Irradiance [??] -9h,Irradiance [??] -10h,Irradiance [??] -11h,Irradiance [??] -12h,Hours,sin(hour),cos(hour),cos(hour) +1h,cos(hour) +2h,cos(hour) +3h,cos(hour) +4h,cos(hour) +5h,cos(hour) +6h,cos(hour) +7h,cos(hour) +8h,cos(hour) +9h,Day of the Year,neg cos(DoY),Day of the Week,Month,Week of the Year,Weekend Flag,Working Hour,Summer Holidays,Night Time,Deep Night,Early Morning
2014-01-17 18:00:00,0.0,55.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.35,60.90,61.24,61.10,59.64,58.98,3.186133,3.912433,4.188183,4.109800,3.706867,2.988300,1.983517,0.730083,0.042583,0.000000,18,-1.000000,-1.836970e-16,0.258819,0.500000,0.707107,0.866025,0.965926,1.000000,0.965926,8.660254e-01,7.071068e-01,17,-0.99763,5,1,3,0,1,0,1,0,0
2014-01-17 19:00:00,0.0,51.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.42,58.35,60.90,61.24,61.10,59.64,1.910283,3.186133,3.912433,4.188183,4.109800,3.706867,2.988300,1.983517,0.730083,0.042583,19,-0.965926,2.588190e-01,0.500000,0.707107,0.866025,0.965926,1.000000,0.965926,0.866025,7.071068e-01,5.000000e-01,17,-0.99763,5,1,3,0,0,0,1,0,0
2014-01-17 20:00:00,0.0,47.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.24,55.42,58.35,60.90,61.24,61.10,0.339100,1.910283,3.186133,3.912433,4.188183,4.109800,3.706867,2.988300,1.983517,0.730083,20,-0.866025,5.000000e-01,0.707107,0.866025,0.965926,1.000000,0.965926,0.866025,0.707107,5.000000e-01,2.588190e-01,17,-0.99763,5,1,3,0,0,0,1,0,0
2014-01-17 21:00:00,0.0,43.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.29,51.24,55.42,58.35,60.90,61.24,0.000000,0.339100,1.910283,3.186133,3.912433,4.188183,4.109800,3.706867,2.988300,1.983517,21,-0.707107,7.071068e-01,0.866025,0.965926,1.000000,0.965926,0.866025,0.707107,0.500000,2.588190e-01,6.123234e-17,17,-0.99763,5,1,3,0,0,0,1,0,0
2014-01-17 22:00:00,0.0,42.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.89,47.29,51.24,55.42,58.35,60.90,0.000000,0.000000,0.339100,1.910283,3.186133,3.912433,4.188183,4.109800,3.706867,2.988300,22,-0.500000,8.660254e-01,0.965926,1.000000,0.965926,0.866025,0.707107,0.500000,0.258819,6.123234e-17,-2.588190e-01,17,-0.99763,5,1,3,0,0,0,1,0,0


## Correlations

In [166]:
DF_mod.corr() 

,Consumption [W],Temperature [°K],Irradiance [??],Consumption [W] -1h,Consumption [W] -2h,Consumption [W] -3h,Consumption [W] -4h,Consumption [W] -5h,Consumption [W] -6h,Consumption [W] -7h,Consumption [W] -8h,Consumption [W] -9h,Consumption [W] -10h,Consumption [W] -11h,Consumption [W] -12h,Consumption [W] -13h,Consumption [W] -14h,Consumption [W] -15h,Consumption [W] -16h,Consumption [W] -17h,Consumption [W] -18h,Consumption [W] -19h,Consumption [W] -20h,Consumption [W] -21h,Consumption [W] -22h,Consumption [W] -23h,Consumption [W] -24h,Temperature [°K] -1h,Temperature [°K] -2h,Temperature [°K] -3h,Temperature [°K] -4h,Temperature [°K] -5h,Temperature [°K] -6h,Irradiance [??] -3h,Irradiance [??] -4h,Irradiance [??] -5h,Irradiance [??] -6h,Irradiance [??] -7h,Irradiance [??] -8h,Irradiance [??] -9h,Irradiance [??] -10h,Irradiance [??] -11h,Irradiance [??] -12h,Hours,sin(hour),cos(hour),cos(hour) +1h,cos(hour) +2h,cos(hour) +3h,cos(hour) +4h,cos(hour) +5h,cos(hour) +6h,cos(hour) +7h,cos(hour) +8h,cos(hour) +9h,Day of the Year,neg cos(DoY),Day of the Week,Month,Week of the Year,Weekend Flag,Working Hour,Summer Holidays,Night Time,Deep Night,Early Morning
Consumption [W],1.000000,0.568045,-0.012301,0.906876,0.810004,0.693165,0.566553,0.433470,0.302756,0.181403,0.077110,-0.007536,-0.071397,-0.112616,-0.128299,-0.112829,-0.072357,-0.008803,0.076106,0.178017,0.297281,0.422111,0.549065,0.666633,0.768909,0.848761,0.899489,0.608405,0.637044,0.650513,0.648243,0.630598,0.598720,0.368102,0.471774,0.538318,0.559699,0.535718,0.473295,0.386039,0.276353,0.157148,0.038935,0.364432,-0.442416,0.213078,0.320201,0.405567,0.463366,0.489657,0.482640,0.442783,0.372794,0.277443,0.163236,0.129344,0.505276,-0.017835,0.129259,0.130640,-0.010021,-0.052747,0.454866,0.125182,-0.125182,-0.326873
Temperature [°K],0.568045,1.000000,0.355214,0.520306,0.469019,0.417294,0.368703,0.326146,0.290875,0.264893,0.248928,0.242617,0.247482,0.264976,0.294523,0.334495,0.381984,0.433629,0.486375,0.534439,0.574393,0.603571,0.619074,0.620144,0.606057,0.577316,0.537596,0.990902,0.968113,0.934783,0.894327,0.850000,0.804708,0.453268,0.425452,0.374388,0.307202,0.230550,0.150924,0.073879,0.004151,-0.056185,-0.106377,0.238367,-0.322094,-0.170424,-0.081287,0.013422,0.107227,0.193721,0.266999,0.322075,0.355191,0.364084,0.348165,0.168526,0.770686,0.030848,0.168486,0.171836,0.028941,0.266132,0.528015,0.203247,-0.203247,-0.117973
Irradiance [??],-0.012301,0.355214,1.000000,-0.120039,-0.206377,-0.269383,-0.310981,-0.331162,-0.330680,-0.311301,-0.276331,-0.226043,-0.158524,-0.072552,0.030914,0.147982,0.263910,0.371183,0.456201,0.514685,0.536876,0.514687,0.447074,0.346635,0.222214,0.093288,-0.024982,0.268265,0.169853,0.069486,-0.024070,-0.103658,-0.164542,0.561132,0.330127,0.104690,-0.094077,-0.252622,-0.363610,-0.429399,-0.460499,-0.471648,-0.473565,0.169120,-0.239571,-0.740865,-0.653741,-0.521953,-0.354476,-0.162769,0.040009,0.240023,0.423562,0.578121,0.693123,-0.028258,0.108345,-0.030367,-0.029046,-0.026408,-0.030880,0.767481,0.073343,0.398514,-0.398514,0.169041
Consumption [W] -1h,0.906876,0.520306,-0.120039,1.000000,0.906876,0.810004,0.693165,0.566553,0.433470,0.302756,0.181403,0.077110,-0.007536,-0.071397,-0.112616,-0.128299,-0.112829,-0.072357,-0.008803,0.076106,0.178017,0.297281,0.422111,0.549065,0.666633,0.768909,0.848761,0.568104,0.608468,0.637110,0.650577,0.648308,0.630655,0.240308,0.367835,0.471447,0.537984,0.559418,0.535530,0.473203,0.386015,0.276359,0.157208,0.327986,-0.372261,0.320258,0.405567,0.463305,0.489540,0.482483,0.442603,0.372607,0.277261,0.163066,0.037814,0.129679,0.505134,-0.019495,0.129622,0.131008,-0.012949,-0.157857,0.454794,0.049165,-0.049165,-0.339030
Consumption [W] -2h,0.810004,0.469019,-0.206377,0.906876,1.000000,0.906876,0.810004,0.693165,0.566553,0.433470,0.302756,0.181403,0.077110,-0.007536,-0.071397,-0.112616,-0.128299,-0.112829,-0.072357,-0.008803,0.076106,0.178017,0.297281,0.422111,0.549065,0.666633,0.768909,0.520351,0.568155,0.608526,0.637170,0.6506

## Machine Learning

In [167]:
DF_mod_filtered = DF_mod.loc["2014-03-01":"2014-12-29"]

In [168]:
DF_target = DF_mod_filtered["Consumption [W]"]
DF_features = DF_mod_filtered.drop("Consumption [W]",axis=1)

### Linear Regression

In [202]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(DF_features,DF_target,test_size=0.15)

from sklearn import linear_model
linear_reg = linear_model.LinearRegression()

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [170]:
linear_reg.fit(x_train,y_train)
predicted_linearReg_split = linear_reg.predict(x_test)

MAE_linearReg_split = mean_absolute_error(predicted_linearReg_split,y_test)
MSE_linearReg_split = mean_squared_error(predicted_linearReg_split,y_test)
R2_linearReg_split = r2_score(predicted_linearReg_split,y_test)
print("MAE: "+str(MAE_linearReg_split)+" MSE: "+str(MSE_linearReg_split)+" R2: "+str(R2_linearReg_split))

MAE: 130.30611377320892 MSE: 51272.22021020299 R2: 0.8835515555653429


In [171]:
predicted_DF_linearReg_split = pd.DataFrame(predicted_linearReg_split,index=y_test.index,columns=["AC_predicted_linearReg_split"])
predicted_DF_linearReg_joined = predicted_DF_linearReg_split.join(y_test)
predicted_DF_linearReg_joined_august = predicted_DF_linearReg_joined.loc["2014-08-01":"2014-09-01"]

### K-fold Cross Validation

In [172]:
from sklearn.model_selection import cross_val_predict

In [173]:
predict_linearReg_CV = cross_val_predict(linear_reg,DF_features,DF_target,cv=40)

MAE_linearReg_CV = mean_absolute_error(predict_linearReg_CV,DF_target)
MSE_linearReg_CV = mean_squared_error(predict_linearReg_CV,DF_target)
R2_linearReg_CV = r2_score(predict_linearReg_CV,DF_target)
print("MAE: "+str(MAE_linearReg_CV)+" MSE: "+str(MSE_linearReg_CV)+" R2: "+str(R2_linearReg_CV))

MAE: 137.3657777466078 MSE: 52845.60408195664 R2: 0.8792105394525548


In [174]:
predicted_DF_linearReg_CV = pd.DataFrame(predict_linearReg_CV,index=DF_target.index,columns=["AC_predicted_linearReg_CV"])
predicted_DF_linearReg_CV_joined = predicted_DF_linearReg_CV.join(DF_target)
predicted_DF_linearReg_CV_joined_august = predicted_DF_linearReg_CV_joined.loc["2014-08-01":"2014-09-01"]

### Random Forest

In [175]:
from sklearn.ensemble import RandomForestRegressor
reg_RF = RandomForestRegressor () 

In [176]:
from sklearn.model_selection import cross_val_predict
predict_RF_CV = cross_val_predict(reg_RF,DF_features,DF_target,cv=10)
MAE_RF_CV = mean_absolute_error(predict_RF_CV,DF_target)
MSE_RF_CV = mean_squared_error(predict_RF_CV,DF_target)
R2_RF_CV = r2_score(predict_RF_CV,DF_target)
print("MAE: "+str(MAE_RF_CV)+" MSE: "+str(MSE_RF_CV)+" R2: "+str(R2_RF_CV))

MAE: 99.54338351119081 MSE: 40392.44672384492 R2: 0.9076944833545614


In [177]:
predicted_DF_RF_CV = pd.DataFrame(predict_RF_CV,index=DF_target.index,columns=["AC_predicted_linearReg_CV"])
predicted_DF_RF_CV_joined = predicted_DF_RF_CV.join(DF_target)
predicted_DF_RF_CV_joined_august = predicted_DF_RF_CV_joined.loc ["2014-08-01":"2014-09-01"]

## Improving Accuracy

In [206]:
def R2Accuracy (DataFrame,Partition,CrossValidateFolds):
  Target = DataFrame["Consumption [W]"]
  Features = DataFrame.drop("Consumption [W]",axis=1)
  x_train, x_test, y_train, y_test = train_test_split(Features,Target,test_size=Partition)
  Linear_Regression = linear_model.LinearRegression()
  Predict_LinearReg_CV = cross_val_predict(Linear_Regression,Features,Target,cv=CrossValidateFolds)
  R2_CV= r2_score(Predict_LinearReg_CV,Target)
  return R2_CV

In [180]:
def ColumnRemover(DataFrame):
  Temp = DataFrame.drop("Consumption [W]",axis=1)
  Columns = Temp.columns
  DF_Test = DataFrame
  DF_Final = DataFrame
  Biggest_R2 = R2Accuracy(DataFrame,0.15,40)
  for i in Columns:
    DF_Test = DF_Final.drop(columns=i,inplace=False)
    Test_R2 = R2Accuracy(DF_Test,0.15,40)
    if Test_R2 < Biggest_R2:
      DF_Test = DF_Final
    else:
      DF_Final = DF_Test
  return DF_Final

In [181]:
Test = DF_mod_filtered
Result = ColumnRemover(Test)
Result.head()

,Consumption [W],Consumption [W] -1h,Consumption [W] -2h,Consumption [W] -6h,Consumption [W] -12h,Consumption [W] -20h,Consumption [W] -23h,Consumption [W] -24h,Temperature [°K] -2h,Irradiance [??] -4h,Irradiance [??] -8h,Working Hour,Deep Night,Early Morning
2014-03-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.53,0.0,2.782150,0,1,0
2014-03-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.61,0.0,1.334650,0,1,0
2014-03-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.88,0.0,0.060583,0,1,0
2014-03-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.80,0.0,0.000000,0,1,0
2014-03-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.14,0.0,0.000000,0,1,0


## Regressor 1: PLSRegression

In [182]:
from sklearn.cross_decomposition import PLSRegression
PLS_Reg = PLSRegression()
PLS_Reg.fit(x_train,y_train)
predicted_PLS_Reg = PLS_Reg.predict(x_test)

MAE_PLS_Reg = mean_absolute_error(predicted_PLS_Reg,y_test)
MSE_PLS_Reg = mean_squared_error(predicted_PLS_Reg,y_test)
R2_PLS_Reg = r2_score(predicted_PLS_Reg,y_test)
print("MAE: "+str(MAE_PLS_Reg)+" MSE: "+str(MSE_PLS_Reg)+" R2: "+str(R2_PLS_Reg))

MAE: 214.43407684318058 MSE: 84491.33902464207 R2: 0.7876060831016181


In [183]:
predicted_DF_PLS_Reg = pd.DataFrame(predicted_PLS_Reg,index=y_test.index,columns=["AC_PLSReg_split"])
predicted_DF_PLS_Reg_joined = predicted_DF_PLS_Reg.join(y_test)
predicted_DF_PLS_Reg_joined_august = predicted_DF_PLS_Reg_joined.loc["2014-08-01":"2014-09-01"]

### Regressor 2: Multi-Layer Perceptron Regressor

In [184]:
from sklearn.neural_network import MLPRegressor
MLP_Reg = MLPRegressor()
MLP_Reg.fit(x_train,y_train)
predicted_MLP_Reg = MLP_Reg.predict(x_test)

MAE_MLP_Reg = mean_absolute_error(predicted_MLP_Reg,y_test)
MSE_MLP_Reg = mean_squared_error(predicted_MLP_Reg,y_test)
R2_MLP_Reg = r2_score(predicted_MLP_Reg,y_test)
print("MAE: "+str(MAE_MLP_Reg)+" MSE: "+str(MSE_MLP_Reg)+" R2: "+str(R2_MLP_Reg))

MAE: 113.0960178399594 MSE: 44601.37733368453 R2: 0.898339956703133


In [185]:
predicted_DF_MLP_Reg = pd.DataFrame(predicted_MLP_Reg,index=y_test.index,columns=["AC_MLPReg_split"])
predicted_DF_MLP_Reg_joined = predicted_DF_MLP_Reg.join(y_test)
predicted_DF_MLP_Reg_joined_august = predicted_DF_MLP_Reg_joined.loc["2014-08-01":"2014-09-01"]

### Regressor 3: Decision Tree Regressor

In [186]:
from sklearn.tree import DecisionTreeRegressor
DT_Reg = DecisionTreeRegressor()
DT_Reg.fit(x_train,y_train)
predicted_DT_Reg = DT_Reg.predict(x_test)

MAE_DT_Reg = mean_absolute_error(predicted_DT_Reg,y_test)
MSE_DT_Reg = mean_squared_error(predicted_DT_Reg,y_test)
R2_DT_Reg = r2_score(predicted_DT_Reg,y_test)
print("MAE: "+str(MAE_DT_Reg)+" MSE: "+str(MSE_DT_Reg)+" R2: "+str(R2_DT_Reg))

MAE: 116.43766543560051 MSE: 68570.22592705507 R2: 0.8609279886886494


In [187]:
predicted_DF_DT_Reg = pd.DataFrame(predicted_DT_Reg,index=y_test.index,columns=["AC_DTReg_split"])
predicted_DF_DT_Reg_joined = predicted_DF_DT_Reg.join(y_test)
predicted_DF_DT_Reg_joined_august = predicted_DF_DT_Reg_joined.loc["2014-08-01":"2014-09-01"]